In [1]:
#Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

#Reading in data
df = pd.read_csv('Resources/ev_stations_v1.csv', low_memory=False)
df.head()

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Pricing (French),LPG Nozzle Types,Hydrogen Pressures,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Access
0,ELEC,LADWP - Truesdale Center,11797 Truesdale St,NaN,Sun Valley,CA,91352,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ELEC,LADWP - West LA District Office,1394 S Sepulveda Blvd,NaN,Los Angeles,CA,90024,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ELEC,Los Angeles Convention Center,1201 S Figueroa St,West hall and South hall,Los Angeles,CA,90015,NaN,213-741-1151,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ELEC,LADWP - John Ferraro Building,111 N Hope St,Across Hope,Los Angeles,CA,90012,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ELEC,LADWP - Haynes Power Plant,6801 E 2nd St,NaN,Long Beach,CA,90803,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
df.columns

Index(['Fuel Type Code', 'Station Name', 'Street Address',
       'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
       'BD Blends', 'NG Fill Type Code', 'NG PSI', 'EV Level1 EVSE Num',
       'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Other Info', 'EV Network',
       'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude',
       'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code',
       'Federal Agency ID', 'Federal Agency Name', 'Open Date',
       'Hydrogen Status Link', 'NG Vehicle Class', 'LPG Primary',
       'E85 Blender Pump', 'EV Connector Types', 'Country',
       'Intersection Directions (French)', 'Access Days Time (French)',
       'BD Blends (French)', 'Groups With Access Code (French)',
       'Hydrogen Is Retail', 'Access Code', 'Access Detail Code',
       'Federal Agency Code', 'Facility Type', 'CNG Dispenser N

In [3]:
df['Open Date']

0        1999-10-15
1        2020-02-28
2        1995-08-30
3        1999-10-15
4        2018-05-01
            ...    
50284    2022-01-14
50285    2022-01-14
50286    2022-01-14
50287    2022-01-14
50288    2022-01-14
Name: Open Date, Length: 50289, dtype: object

In [4]:
#Removing unecessary columns
stations = df[['Station Name', 
                'Street Address', 
                'City', 
                'State', 
                'ZIP', 
                'Access Code', 
                'EV Network', 
                'EV Connector Types', 
                'Latitude', 
                'Longitude',
                'Open Date']]

#stations.head()
stations.shape

(50289, 11)

In [5]:
#Removing null values
stations = stations.dropna()

#stations.head()
stations.shape

(50168, 11)

In [6]:
#Filtering data for only stations in the US (including Washington DC and Puerto Rico)
stations = stations[stations['State'] != 'ON']
#Removing 'ON' or Ontario

#stations['State'].unique()
#---------------------------------------------------------------------------
#output:
#array(['CA', 'VT', 'WA', 'OR', 'IL', 'ID', 'TX', 'TN', 'NY', 'FL', 'WI',
#       'IA', 'MN', 'AR', 'AZ', 'CT', 'MD', 'NJ', 'SC', 'MI', 'OH', 'WV',
#       'MO', 'UT', 'KS', 'MA', 'CO', 'IN', 'LA', 'NC', 'NH', 'VA', 'AL',
#       'GA', 'HI', 'NV', 'DC', 'PA', 'RI', 'OK', 'ME', 'KY', 'NE', 'MS',
#       'SD', 'DE', 'NM', 'MT', 'ND', 'WY', 'AK', 'PR', 'ON'], dtype=object)

len(stations['State'].unique())

52

In [7]:
#Renaming columns
rename = {'Station Name':'station_name', 
            'Street Address':'address', 
            'City':'city', 
            'State':'state', 
            'ZIP':'zip',
            'Access Code':'access', 
            'EV Network':'ev_network', 
            'EV Connector Types':'connector_type',
            'Latitude':'lat',
            'Longitude':'lng',
            'Open Date':'open_date'}

stations = stations.rename(mapper=rename, axis=1)

#stations.head()
stations.columns

Index(['station_name', 'address', 'city', 'state', 'zip', 'access',
       'ev_network', 'connector_type', 'lat', 'lng', 'open_date'],
      dtype='object')

In [8]:
# Resetting index
stations = stations.reset_index()
stations.head()

,index,station_name,address,city,state,zip,access,ev_network,connector_type,lat,lng,open_date
0,0,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,private,Non-Networked,CHADEMO J1772 J1772COMBO,34.248319,-118.387971,1999-10-15
1,1,LADWP - West LA District Office,1394 S Sepulveda Blvd,Los Angeles,CA,90024,private,Non-Networked,J1772,34.052542,-118.448504,2020-02-28
2,2,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,public,Non-Networked,J1772,34.040539,-118.271387,1995-08-30
3,3,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,private,Non-Networked,CHADEMO J1772 J1772COMBO,34.059133,-118.248589,1999-10-15
4,4,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,90803,private,Non-Networked,CHADEMO J1772 J1772COMBO,33.759802,-118.096665,2018-05-01


In [9]:
# Deleting extra index column
del stations['index']
stations.head()

,station_name,address,city,state,zip,access,ev_network,connector_type,lat,lng,open_date
0,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,private,Non-Networked,CHADEMO J1772 J1772COMBO,34.248319,-118.387971,1999-10-15
1,LADWP - West LA District Office,1394 S Sepulveda Blvd,Los Angeles,CA,90024,private,Non-Networked,J1772,34.052542,-118.448504,2020-02-28
2,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,public,Non-Networked,J1772,34.040539,-118.271387,1995-08-30
3,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,private,Non-Networked,CHADEMO J1772 J1772COMBO,34.059133,-118.248589,1999-10-15
4,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,90803,private,Non-Networked,CHADEMO J1772 J1772COMBO,33.759802,-118.096665,2018-05-01


In [10]:
#Creating engine to sqlite
engine = create_engine('sqlite:///Resources/evstations.sqlite', echo=False)

#Exporting to sqlite
stations.to_sql(name='evstations', con=engine, if_exists='replace', index=True)

In [11]:
#Exporting to csv
stations.to_csv('Resources/evstations_clean.csv')

In [12]:
stations.head()

,station_name,address,city,state,zip,access,ev_network,connector_type,lat,lng,open_date
0,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,private,Non-Networked,CHADEMO J1772 J1772COMBO,34.248319,-118.387971,1999-10-15
1,LADWP - West LA District Office,1394 S Sepulveda Blvd,Los Angeles,CA,90024,private,Non-Networked,J1772,34.052542,-118.448504,2020-02-28
2,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,public,Non-Networked,J1772,34.040539,-118.271387,1995-08-30
3,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,private,Non-Networked,CHADEMO J1772 J1772COMBO,34.059133,-118.248589,1999-10-15
4,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,90803,private,Non-Networked,CHADEMO J1772 J1772COMBO,33.759802,-118.096665,2018-05-01
